# Purchasing Analytics

In this notebook, I will analyse the purchasing behavior of the customers.

The dataset we are going to use is a dataset of each transaction, rather than the customer from the last one.
The dataset consists of 500 individuals and their purchase of candy bars from the same store over a period of 2 years.

Here are the Legend:

![](Legend1.png)

![](Legend2.png)